In [1]:
import coiled
import dask
import dask.dataframe as dd

In [2]:
cluster = coiled.Cluster(name="pow-cluster", n_workers=10)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/rich/live.py:22
1: UserWarning: install "ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')

Found software environment build
Created fw rule: inbound [8786-8787] [0.0.0.0/0] []
Created FW rules: coiled-dask-matthew24-142549-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-142549-firewall -> coiled-dask-matthew24-142549-firewall]
Created FW rules: coiled-dask-matthew24-142549-cluster-firewall
Created fw rule: cluster [0-65535] [None] [coiled-dask-matthew24-142549-cluster-firewall -> coiled-dask-matthew24-142549-cluster-firewall]
Created scheduler VM: coiled-dask-matthew24-142549-scheduler (type: t3a.medium, ip: ['54.236.57.232'])


In [3]:
client = dask.distributed.Client(cluster)

/Users/powers/opt/miniconda3/envs/standard-coiled/lib/python3.9/site-packages/distributed/client.py:1265: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | client         | scheduler      | workers        |
+---------+----------------+----------------+----------------+
| lz4     | 4.0.0          | 3.1.10         | 3.1.10         |
| numpy   | 1.22.3         | 1.21.5         | 1.21.5         |
| pandas  | 1.4.2          | 1.4.0          | 1.4.0          |
| python  | 3.9.12.final.0 | 3.9.10.final.0 | 3.9.10.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


## Query with Parquet

In [6]:
ddf = dd.read_parquet(
    "s3://coiled-datasets/timeseries/20-years/parquet",
    storage_options={"anon": True, "use_ssl": True},
    engine="pyarrow",
    columns=["name"],
)

In [7]:
%%time
ddf["name"].nunique().compute()

CPU times: user 192 ms, sys: 34.1 ms, total: 226 ms
Wall time: 26.5 s


26

## Same query with JSON

In [4]:
ddf = dd.read_json(
    "s3://coiled-datasets/timeseries/20-years/json/*.part",
    storage_options={"anon": True, "use_ssl": True},
    lines=True,
)

In [5]:
%%time
ddf["name"].nunique().compute()

CPU times: user 1.43 s, sys: 124 ms, total: 1.55 s
Wall time: 3min 53s


26